<h1>MODELS(COVID PROJECT)</h1>
<ul>
    <li>This Notebook contains the formation of models on both the datasets</li>
    <li>The Datasets have undergone through various image processing techniques done in Lung Segmentation with Documentation.ipynb</li>

In [33]:
import os
import cv2
import keras
import numpy as np
import tensorflow as tf
from keras.utils.np_utils import to_categorical
from keras.models import load_model  
#from keras.utils import plot_model
from keras import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Softmax,Activation,Dense,Dropout
from keras.callbacks import Callback,ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,auc
#from skimage.transform import rescale, resize
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
import pickle
from skimage import measure
from skimage import morphology
from skimage.transform import resize
from sklearn.cluster import KMeans
from sklearn.metrics import roc_auc_score

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.models import load_model 
from keras.utils.vis_utils import plot_model
import cv2
import os
from sklearn.model_selection import train_test_split

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from learntools.core import binder
binder.bind(globals())
from learntools.computer_vision.ex2 import *
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import EfficientNetB1
from keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.densenet  import DenseNet169
from keras.layers import LSTM
from keras.layers import TimeDistributed

In [4]:
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
import cv2
import glob

<h3>Image Data Generator</h3>
<ol>
    <li>Used for Loading the datasets</li>
    <li>Perform Techniques like Pixel centering, Pixel Normalization and Pixel Standardization</li>
    <li>Validation Split is only used in the first dataset. It is a 70-30 split</li>
</ol>

<h4>Dataset 1</h4>

In [5]:
covid_dir='../input/covid-project/Image Dataset 2/IMAGES_NoHist13000'
SIZE_X=SIZE_Y=224
BATCH_SIZE = 128
from keras.preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator(rescale=1.0/255.0,
                           featurewise_center=True,
                           featurewise_std_normalization=True,
                           validation_split = 0.3)
training=datagen.flow_from_directory(covid_dir,
                                  class_mode = "categorical",
                                  target_size = (SIZE_X,SIZE_Y),
                                  color_mode="rgb",
                                  batch_size = 128, 
                                  shuffle = False,
                                  subset='training',
                                  seed = 42)

validation=datagen.flow_from_directory(covid_dir,
                                    class_mode = "categorical",
                                    target_size = (SIZE_X,SIZE_Y),
                                    color_mode="rgb",
                                    batch_size = 128, 
                                    shuffle = False,
                                    subset='validation',
                                    seed = 42)

<h4>Dataset 2</h4>

In [6]:
covid_dir='../input/covid-project/Image Dataset 2/Images_NoHist2000'
SIZE_X=SIZE_Y=224
BATCH_SIZE = 128
from keras.preprocessing.image import ImageDataGenerator
datagen2=ImageDataGenerator(rescale=1.0/255.0,
                           featurewise_center=True,
                           featurewise_std_normalization=True)
testing=datagen2.flow_from_directory(covid_dir,
                                  class_mode = "categorical",
                                  target_size = (SIZE_X,SIZE_Y),
                                  color_mode="rgb",
                                  batch_size = 128, 
                                  shuffle = False,
                                  #subset='training',
                                  seed = 42)

<h3>Models</h3><br>
<ol>
    <li>Weights Are Loaded From Keras Library</li>
    <li>All the models undergo Transfer Learning and Fine tuning</li>
    <li>The Head of The model is changed according to the datasets</li>
    <li>The best weights of the model are stored</li>
    <li>The model is trained and Validated on the first dataset and using the weights the model has learnt from the first dataset,it tests its results on the second dataset</li>
    <li>Confusion Matrix,Classification report and Roc curves are used to test and Compare the accuracies of the models</li>
</ol>

<h4>VGG16</h4>

In [8]:
def create_model(input_shape,n_classes,optimizer='adam',fine_tune=0):
    conv_base = VGG16(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False
    
    model = Sequential()
    model.add(conv_base)
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer = 'adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [9]:
input_shape=(224,224,3)
optim_1 = Adam(lr=0.0001)
n_classes=2

n_steps=training.samples//128
n_val_steps=validation.samples//128
n_epochs=10

vggmodel=create_model(input_shape,n_classes,optim_1,fine_tune=2)

In [10]:
vggmodel.summary()

In [ ]:
vgg_checkpoint = ModelCheckpoint(filepath='cnn+vggmodel(revised2)_new.weights.best.hdf5',
                                  save_best_only=True,
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='val_accuracy',
                           patience=7,
                           restore_best_weights=True,
                           mode='max')
vgg_history = vggmodel.fit(training,
                           batch_size=BATCH_SIZE,
                           epochs=n_epochs,
                           validation_data=(validation),
                           steps_per_epoch=n_steps,
                           validation_steps=n_val_steps,
                           callbacks=[vgg_checkpoint, early_stop],
                           verbose=1)

In [12]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
vggmodel.load_weights('../input/cnn-model/cnnvggmodel(revised)_new.weights.best (4).hdf5') # initialize the best trained weights
true_classes = testing.classes
vgg_preds = vggmodel.predict(testing)
vgg_pred_classes = np.argmax(vgg_preds, axis=1)
class_indices = testing.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())
target= list(i for i in class_indices.values())
print(classification_report(true_classes,vgg_pred_classes, target_names=target))
vgg_acc = accuracy_score(true_classes, vgg_pred_classes)*100
print("VGG16 Accuracy Dataset 2: {:.2f} %".format(vgg_acc))

In [13]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
vggmodel.load_weights('../input/cnn-model/cnnvggmodel(revised)_new.weights.best (4).hdf5') # initialize the best trained weights
true_classes2 = training.classes
vgg_preds = vggmodel.predict(training)
vgg_pred_classes2 = np.argmax(vgg_preds, axis=1)
class_indices =training.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())
target= list(i for i in class_indices.values())
print(classification_report(true_classes2,vgg_pred_classes2, target_names=target))
vgg_acc = accuracy_score(true_classes2, vgg_pred_classes2)*100
print("VGG16 Accuracy Dataset1(Training) : {:.2f} %".format(vgg_acc))

In [14]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
vggmodel.load_weights('../input/cnn-model/cnnvggmodel(revised)_new.weights.best (4).hdf5') # initialize the best trained weights
true_classes = validation.classes
vgg_preds = vggmodel.predict(validation)
vgg_pred_classes = np.argmax(vgg_preds, axis=1)
class_indices = validation.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())
target= list(i for i in class_indices.values())
print(classification_report(true_classes,vgg_pred_classes, target_names=target))
vgg_acc = accuracy_score(true_classes, vgg_pred_classes)*100
print("VGG16 Accuracy Dataset1(Validation) : {:.2f} %".format(vgg_acc))

<h5>Confusion Matrix</h5>

In [15]:
from sklearn.metrics import accuracy_score,confusion_matrix
import seaborn as sns
cm = confusion_matrix(training.classes, np.argmax(vggmodel.predict(training), axis=1))
cm2 = confusion_matrix(validation.classes, np.argmax(vggmodel.predict(validation), axis=1))
cm1 = cm+cm2
cm3 = confusion_matrix(testing.classes, np.argmax(vggmodel.predict(testing), axis=1))
group_names = ["TP","FP","FN","TN"]
group_counts = ["{0:0.0f}".format(value) for value in cm1.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in cm1.flatten()/np.sum(cm1)]
Labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
Labels = np.asarray(Labels).reshape(2,2)
group_names2 = ["TP","FP","FN","TN"]
group_counts2 = ["{0:0.0f}".format(value) for value in cm2.flatten()]
group_percentages2 = ["{0:.2%}".format(value) for value in cm2.flatten()/np.sum(cm2)]
Labels2 = [f"{v11}\n{v22}\n{v33}" for v11, v22, v33 in zip(group_names2,group_counts2,group_percentages2)]
Labels2 = np.asarray(Labels2).reshape(2,2)
group_names3 = ["TP","FP","FN","TN"]
group_counts3 = ["{0:0.0f}".format(value) for value in cm3.flatten()]
group_percentages3 = ["{0:.2%}".format(value) for value in cm3.flatten()/np.sum(cm3)]
Labels3 = [f"{v111}\n{v222}\n{v333}" for v111, v222, v333 in zip(group_names3,group_counts3,group_percentages3)]
Labels3 = np.asarray(Labels3).reshape(2,2)
fig,[ax1,ax2] = plt.subplots(nrows=1,ncols=2,figsize=(10,5),constrained_layout = True)
sns.heatmap(cm1, annot=Labels, fmt='', cmap='Reds',ax=ax1)
ax1.set_xlabel('Predicted labels')
ax1.set_ylabel('True labels')
ax1.set_title("Dataset1")
ax1.xaxis.set_ticklabels([target[0], target[1]]); ax1.yaxis.set_ticklabels([target[0], target[1]])
sns.heatmap(cm3, annot=Labels3, fmt='', cmap='Blues',ax=ax2)
ax2.set_xlabel('Predicted labels')
ax2.set_ylabel('True labels')
ax2.set_title("Dataset2")
ax2.xaxis.set_ticklabels([target[0], target[1]]); ax2.yaxis.set_ticklabels([target[0], target[1]])
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.7, 
                    hspace=0.7)
#fig.suptitle("This Main Title is Nicely Formatted", fontsize=16)
#fig.suptitle('VGG16+CNN Results(Confusion Matrix)')
#plt.tight_layout()
fig.show()

<h5>ROC Curve</h5>

In [16]:
test_labels = training.classes
results = np.argmax(vggmodel.predict(training), axis=1)
test_labels2 = validation.classes
results2 = np.argmax(vggmodel.predict(validation), axis=1)
test_labels3 = testing.classes
results3 = np.argmax(vggmodel.predict(testing), axis=1)
from scipy import interp
from itertools import cycle
from sklearn.metrics import roc_curve
lw = 2

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(1):
    fpr[i], tpr[i], _ = roc_curve(results,test_labels)
    roc_auc[i] = auc(fpr[i], tpr[i])

fpr2 = dict()
tpr2 = dict()
roc_auc2 = dict()
for i in range(1):
    fpr2[i], tpr2[i], _ = roc_curve(results2,test_labels2)
    roc_auc2[i] = auc(fpr2[i], tpr2[i])
    
fpr3 = dict()
tpr3 = dict()
roc_auc3 = dict()
for i in range(1):
    fpr3[i], tpr3[i], _ = roc_curve(results3,test_labels3)
    roc_auc3[i] = auc(fpr3[i], tpr3[i])


colors = cycle(['red', 'red'])
for i, color in zip(range(1), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='D1:Training (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))
colors2 = cycle(['blue', 'blue'])
for i, color in zip(range(1), colors2):
    plt.plot(fpr2[i], tpr2[i], color=color, lw=lw,
             label='D1:Validation (area = {1:0.2f})'
             ''.format(i, roc_auc2[i]))
colors3 = cycle(['orange', 'orange'])
for i, color in zip(range(1), colors3):
    plt.plot(fpr3[i], tpr3[i], color=color, lw=lw,
             label='D2 (area = {1:0.2f})'
             ''.format(i, roc_auc3[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve: VGG16')
plt.legend(bbox_to_anchor=(1.8, 1.0), loc='upper right')
plt.show()

<h4>VGG19</h4>

In [17]:
def create_model(input_shape,n_classes,optimizer='adam',fine_tune=0):
    conv_base = VGG19(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False
    
    model = Sequential()
    model.add(conv_base)
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer = 'adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [18]:
input_shape=(224,224,3)
optim_1 = Adam(lr=0.0001)
n_classes=2

n_steps=training.samples//128
n_val_steps=validation.samples//128
n_epochs=10

vggmodel19=create_model(input_shape,n_classes,optim_1,fine_tune=2)

In [19]:
vggmodel19.summary()

In [ ]:
vgg_checkpoint = ModelCheckpoint(filepath='cnn+vggmodel19(nnew).weights.best.hdf5',
                                  save_best_only=True,
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='val_accuracy',
                           patience=5,
                           restore_best_weights=True,
                           mode='max')

In [ ]:
vgg19_history = vggmodel19.fit(training,
                           batch_size=BATCH_SIZE,
                           epochs=n_epochs,
                           validation_data=(validation),
                           steps_per_epoch=n_steps,
                           validation_steps=n_val_steps,
                           callbacks=[vgg_checkpoint, early_stop],
                           verbose=1)

In [20]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
vggmodel19.load_weights('../input/cnn-model/cnnvggmodel19(nnew).weights.best.hdf5') # initialize the best trained weights
true_classes = testing.classes
vgg_preds = vggmodel19.predict(testing)
vgg_pred_classes = np.argmax(vgg_preds, axis=1)
class_indices = testing.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())
target= list(i for i in class_indices.values())
print(classification_report(true_classes,vgg_pred_classes, target_names=target))
vgg_acc = accuracy_score(true_classes, vgg_pred_classes)*100
print("Vgg19 Accuracy on Dataset2: {:.2f} %".format(vgg_acc))

In [21]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
vggmodel19.load_weights('../input/cnn-model/cnnvggmodel19(nnew).weights.best.hdf5') # initialize the best trained weights
true_classes = training.classes
vgg_preds = vggmodel19.predict(training)
vgg_pred_classes = np.argmax(vgg_preds, axis=1)
class_indices = training.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())
target= list(i for i in class_indices.values())
print(classification_report(true_classes,vgg_pred_classes, target_names=target))
vgg_acc = accuracy_score(true_classes, vgg_pred_classes)*100
print("Vgg19 Accuracy on Dataset1(Training): {:.2f} %".format(vgg_acc))

In [22]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
vggmodel19.load_weights('../input/cnn-model/cnnvggmodel19(nnew).weights.best.hdf5') # initialize the best trained weights
true_classes = validation.classes
vgg_preds = vggmodel19.predict(validation)
vgg_pred_classes = np.argmax(vgg_preds, axis=1)
class_indices = validation.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())
target= list(i for i in class_indices.values())
print(classification_report(true_classes,vgg_pred_classes, target_names=target))
vgg_acc = accuracy_score(true_classes, vgg_pred_classes)*100
print("Vgg19 Accuracy on Dataset1(Validation): {:.2f} %".format(vgg_acc))

<h5>Confusion Matrix</h5>

In [23]:
from sklearn.metrics import accuracy_score,confusion_matrix
import seaborn as sns
cm = confusion_matrix(training.classes, np.argmax(vggmodel19.predict(training), axis=1))
cm2 = confusion_matrix(validation.classes, np.argmax(vggmodel19.predict(validation), axis=1))
cm1 = cm+cm2
cm3 = confusion_matrix(testing.classes, np.argmax(vggmodel19.predict(testing), axis=1))
group_names = ["TP","FP","FN","TN"]
group_counts = ["{0:0.0f}".format(value) for value in cm1.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in cm1.flatten()/np.sum(cm1)]
Labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
Labels = np.asarray(Labels).reshape(2,2)
group_names2 = ["TP","FP","FN","TN"]
group_counts2 = ["{0:0.0f}".format(value) for value in cm2.flatten()]
group_percentages2 = ["{0:.2%}".format(value) for value in cm2.flatten()/np.sum(cm2)]
Labels2 = [f"{v11}\n{v22}\n{v33}" for v11, v22, v33 in zip(group_names2,group_counts2,group_percentages2)]
Labels2 = np.asarray(Labels2).reshape(2,2)
group_names3 = ["TN","FP","FN","TP"]
group_counts3 = ["{0:0.0f}".format(value) for value in cm3.flatten()]
group_percentages3 = ["{0:.2%}".format(value) for value in cm3.flatten()/np.sum(cm3)]
Labels3 = [f"{v111}\n{v222}\n{v333}" for v111, v222, v333 in zip(group_names3,group_counts3,group_percentages3)]
Labels3 = np.asarray(Labels3).reshape(2,2)
fig,[ax1,ax2] = plt.subplots(nrows=1,ncols=2,figsize=(10,5),constrained_layout = True)
sns.heatmap(cm1, annot=Labels, fmt='', cmap='Reds',ax=ax1)
ax1.set_xlabel('Predicted labels')
ax1.set_ylabel('True labels')
ax1.set_title("Dataset1")
ax1.xaxis.set_ticklabels([target[0], target[1]]); ax1.yaxis.set_ticklabels([target[0], target[1]])
sns.heatmap(cm3, annot=Labels3, fmt='', cmap='Blues',ax=ax2)
ax2.set_xlabel('Predicted labels')
ax2.set_ylabel('True labels')
ax2.set_title("Dataset2")
ax2.xaxis.set_ticklabels([target[0], target[1]]); ax2.yaxis.set_ticklabels([target[0], target[1]])
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.7, 
                    hspace=0.7)
#fig.suptitle("This Main Title is Nicely Formatted", fontsize=16)
#fig.suptitle('VGG16+CNN Results(Confusion Matrix)')
#plt.tight_layout()
fig.show()

<h5>Roc Curve</h5>

In [24]:
test_labels = training.classes
results = np.argmax(vggmodel19.predict(training), axis=1)
test_labels2 = validation.classes
results2 = np.argmax(vggmodel19.predict(validation), axis=1)
test_labels3 = testing.classes
results3 = np.argmax(vggmodel19.predict(testing), axis=1)
from scipy import interp
from itertools import cycle
from sklearn.metrics import roc_curve
lw = 2

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(1):
    fpr[i], tpr[i], _ = roc_curve(results,test_labels)
    roc_auc[i] = auc(fpr[i], tpr[i])

fpr2 = dict()
tpr2 = dict()
roc_auc2 = dict()
for i in range(1):
    fpr2[i], tpr2[i], _ = roc_curve(results2,test_labels2)
    roc_auc2[i] = auc(fpr2[i], tpr2[i])
    
fpr3 = dict()
tpr3 = dict()
roc_auc3 = dict()
for i in range(1):
    fpr3[i], tpr3[i], _ = roc_curve(results3,test_labels3)
    roc_auc3[i] = auc(fpr3[i], tpr3[i])

colors = cycle(['red', 'red'])
for i, color in zip(range(1), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='D1:Training (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))
colors2 = cycle(['blue', 'blue'])
for i, color in zip(range(1), colors2):
    plt.plot(fpr2[i], tpr2[i], color=color, lw=lw,
             label='D1:Validation (area = {1:0.2f})'
             ''.format(i, roc_auc2[i]))
colors3 = cycle(['orange', 'orange'])
for i, color in zip(range(1), colors3):
    plt.plot(fpr3[i], tpr3[i], color=color, lw=lw,
             label='D2 (area = {1:0.2f})'
             ''.format(i, roc_auc3[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve: VGG19')
plt.legend(bbox_to_anchor=(1.8, 1.0), loc='upper right')
plt.show()

<h4>Inception V3</h4>

In [25]:
def create_model(input_shape,n_classes,optimizer='adam',fine_tune=0):
    conv_base = InceptionV3(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False
    
    model = Sequential()
    model.add(conv_base)
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer = 'adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [26]:
input_shape=(224,224,3)
optim_1 = Adam(lr=0.0001)
n_classes=2

n_steps=training.samples//128
n_val_steps=validation.samples//128
n_epochs=10

vggmodel=create_model(input_shape,n_classes,optim_1,fine_tune=0)

In [27]:
vggmodel.summary()

In [ ]:
vgg_checkpoint = ModelCheckpoint(filepath='cnn+Inception(revised2)_new.weights.best.hdf5',
                                  save_best_only=True,
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='val_accuracy',
                           patience=7,
                           restore_best_weights=True,
                           mode='max')
vgg_history = vggmodel.fit(training,
                           batch_size=BATCH_SIZE,
                           epochs=n_epochs,
                           validation_data=(validation),
                           steps_per_epoch=n_steps,
                           validation_steps=n_val_steps,
                           callbacks=[vgg_checkpoint, early_stop],
                           verbose=1)

In [28]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
vggmodel.load_weights('../input/cnn-model/cnnInception(revised2)_new.weights.best.hdf5') # initialize the best trained weights
true_classes = testing.classes
vgg_preds = vggmodel.predict(testing)
vgg_pred_classes = np.argmax(vgg_preds, axis=1)
class_indices = testing.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())
target= list(i for i in class_indices.values())
print(classification_report(true_classes,vgg_pred_classes, target_names=target))
vgg_acc = accuracy_score(true_classes, vgg_pred_classes)*100
print("Inception Accuracy Dataset2: {:.2f} %".format(vgg_acc))

In [29]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
vggmodel.load_weights('../input/cnn-model/cnnInception(revised2)_new.weights.best.hdf5') # initialize the best trained weights
true_classes = training.classes
vgg_preds = vggmodel.predict(training)
vgg_pred_classes = np.argmax(vgg_preds, axis=1)
class_indices = training.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())
target= list(i for i in class_indices.values())
print(classification_report(true_classes,vgg_pred_classes, target_names=target))
vgg_acc = accuracy_score(true_classes, vgg_pred_classes)*100
print("Inception Accuracy Dataset1(Training): {:.2f} %".format(vgg_acc))

In [30]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
vggmodel.load_weights('../input/cnn-model/cnnInception(revised2)_new.weights.best.hdf5') # initialize the best trained weights
true_classes = validation.classes
vgg_preds = vggmodel.predict(validation)
vgg_pred_classes = np.argmax(vgg_preds, axis=1)
class_indices = validation.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())
target= list(i for i in class_indices.values())
print(classification_report(true_classes,vgg_pred_classes, target_names=target))
vgg_acc = accuracy_score(true_classes, vgg_pred_classes)*100
print("Inception Accuracy Dataset1(Validation): {:.2f} %".format(vgg_acc))

<h5>Confusion Matrix</h5>

In [31]:
from sklearn.metrics import accuracy_score,confusion_matrix
import seaborn as sns
cm = confusion_matrix(training.classes, np.argmax(vggmodel.predict(training), axis=1))
cm2 = confusion_matrix(validation.classes, np.argmax(vggmodel.predict(validation), axis=1))
cm1 = cm+cm2
cm3 = confusion_matrix(testing.classes, np.argmax(vggmodel.predict(testing), axis=1))
group_names = ["TP","FP","FN","TN"]
group_counts = ["{0:0.0f}".format(value) for value in cm1.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in cm1.flatten()/np.sum(cm1)]
Labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
Labels = np.asarray(Labels).reshape(2,2)
group_names2 = ["TP","FP","FN","TN"]
group_counts2 = ["{0:0.0f}".format(value) for value in cm2.flatten()]
group_percentages2 = ["{0:.2%}".format(value) for value in cm2.flatten()/np.sum(cm2)]
Labels2 = [f"{v11}\n{v22}\n{v33}" for v11, v22, v33 in zip(group_names2,group_counts2,group_percentages2)]
Labels2 = np.asarray(Labels2).reshape(2,2)
group_names3 = ["TP","FP","FN","TN"]
group_counts3 = ["{0:0.0f}".format(value) for value in cm3.flatten()]
group_percentages3 = ["{0:.2%}".format(value) for value in cm3.flatten()/np.sum(cm3)]
Labels3 = [f"{v111}\n{v222}\n{v333}" for v111, v222, v333 in zip(group_names3,group_counts3,group_percentages3)]
Labels3 = np.asarray(Labels3).reshape(2,2)
fig,[ax1,ax2] = plt.subplots(nrows=1,ncols=2,figsize=(10,5),constrained_layout = True)
sns.heatmap(cm1, annot=Labels, fmt='', cmap='Reds',ax=ax1)
ax1.set_xlabel('Predicted labels')
ax1.set_ylabel('True labels')
ax1.set_title("Dataset1")
ax1.xaxis.set_ticklabels([target[0], target[1]]); ax1.yaxis.set_ticklabels([target[0], target[1]])
sns.heatmap(cm3, annot=Labels3, fmt='', cmap='Blues',ax=ax2)
ax2.set_xlabel('Predicted labels')
ax2.set_ylabel('True labels')
ax2.set_title("Dataset2")
ax2.xaxis.set_ticklabels([target[0], target[1]]); ax2.yaxis.set_ticklabels([target[0], target[1]])
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.7, 
                    hspace=0.7)
#fig.suptitle("This Main Title is Nicely Formatted", fontsize=16)
#fig.suptitle('VGG16+CNN Results(Confusion Matrix)')
#plt.tight_layout()
fig.show()

<h5>ROC curve</h5>

In [32]:
test_labels = training.classes
results = np.argmax(vggmodel.predict(training), axis=1)
test_labels2 = validation.classes
results2 = np.argmax(vggmodel.predict(validation), axis=1)
test_labels3 = testing.classes
results3 = np.argmax(vggmodel.predict(testing), axis=1)
from scipy import interp
from itertools import cycle
from sklearn.metrics import roc_curve
lw = 2

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(1):
    fpr[i], tpr[i], _ = roc_curve(results,test_labels)
    roc_auc[i] = auc(fpr[i], tpr[i])

fpr2 = dict()
tpr2 = dict()
roc_auc2 = dict()
for i in range(1):
    fpr2[i], tpr2[i], _ = roc_curve(results2,test_labels2)
    roc_auc2[i] = auc(fpr2[i], tpr2[i])
    
fpr3 = dict()
tpr3 = dict()
roc_auc3 = dict()
for i in range(1):
    fpr3[i], tpr3[i], _ = roc_curve(results3,test_labels3)
    roc_auc3[i] = auc(fpr3[i], tpr3[i])

colors = cycle(['red', 'red'])
for i, color in zip(range(1), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='D1:Training (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))
colors2 = cycle(['blue', 'blue'])
for i, color in zip(range(1), colors2):
    plt.plot(fpr2[i], tpr2[i], color=color, lw=lw,
             label='D1:Validation (area = {1:0.2f})'
             ''.format(i, roc_auc2[i]))
colors3 = cycle(['orange', 'orange'])
for i, color in zip(range(1), colors3):
    plt.plot(fpr3[i], tpr3[i], color=color, lw=lw,
             label='D2 (area = {1:0.2f})'
             ''.format(i, roc_auc3[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve: InceptionV3')
plt.legend(bbox_to_anchor=(1.8, 1.0), loc='upper right')
plt.show()